# Ingest Fantasy Premier League Data into GCP 

This is a notebook for developing etl code for ingesting data from the fantasy.premierleague API into GCP. This API makes available some great free Premier League data related to the Fantasy Football Premier League game. In this notebook, I will go through the process of first steps before scheduling daily updates for certain tables.

### Run the Process Locally

Test your code for downloading the data from the API within your notebook.

In [1]:
from utils.io import dict_from_yaml
from ingest import IngestFpl

In [2]:
config = dict_from_yaml('config.yaml')
ingest_fpl = IngestFpl(config)

### Check Config & Method Help

In [16]:
ingest_fpl.config

{'env': 'dev',
 'api': {'baseurl': 'https://fantasy.premierleague.com/api',
  'endpoints': [{'name': 'bootstrap-static', 'tables': ['elements']},
   {'name': 'fixtures'}]},
 'gcp': {'key_file': 'data-science-on-gcp-323609-78642d80c4ac.json',
  'upload': {'dataset_id': 'fpl_raw',
   'bucketname': 'fpl-staging',
   'tables': {'fixtures': {'file_type': 'json'}}}}}

In [6]:
help(ingest_fpl.run)

Help on method run in module ingest.base:

run(env: str, overrides: dict = None) -> None method of ingest.IngestFpl instance
    Ingestion process runner method to download, parse and upload api data into bigquery
    @param env environment determines which authentication method is used for GCP
    @param overrides overrides for default config, dict with same structure as config



In [4]:
df_dict_raw = ingest_fpl.extract()

In [10]:
df_dict_raw.keys()

dict_keys(['elements', 'fixtures'])

In [5]:
df_dict_raw['fixtures'].head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2292810,1,True,True,1,2022-08-05T19:00:00Z,90,False,True,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,2,74911
1,2292813,1,True,True,4,2022-08-06T11:30:00Z,90,False,True,12,2.0,9,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,2,74914
2,2292811,1,True,True,2,2022-08-06T14:00:00Z,90,False,True,2,0.0,3,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,2,74912
3,2292814,1,True,True,5,2022-08-06T14:00:00Z,90,False,True,20,1.0,11,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,74915
4,2292816,1,True,True,7,2022-08-06T14:00:00Z,90,False,True,16,0.0,15,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,4,74917


In [5]:
df_dict_raw['elements'].head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,100.0,100.0,84450,0,0,-2,2,1,3,4.3,...,265,157,62,32,54,25,120,41,1.06,0.39
1,0.0,0.0,153256,0,0,-4,4,1,3,0.0,...,627,327,507,184,442,193,249,80,0.81,0.00
2,NaN,NaN,156074,0,0,-3,3,0,2,2.3,...,592,163,185,49,398,141,366,138,0.96,0.00
3,100.0,100.0,167199,0,0,-3,3,0,3,1.7,...,311,186,251,120,199,78,277,92,1.03,0.40
4,100.0,100.0,184029,1,-1,2,-2,5,3,9.3,...,46,23,2,2,7,2,9,5,1.06,0.37


# Override Default Tables 

Some of the tables available from the Fantasy Football api do not change week on week, therefore batch loading is not required. For these tables we will override the default config and upload once to bigquery. Set autodetect mode to true to autodetect the schema. Then run a one-off full load of all tables.

[**Note:** The run step requires authentication setup with a service key in your config file, follow the readme for more details]

In [10]:
overrides = {
    'api': {
        'endpoints': [
            {'name': 'bootstrap-static',
            'tables': ['elements', 'element_types', 'teams']},
            {'name': 'fixtures'}
        ]
    },
    'gcp': {
        'upload': {
            'autodetect_mode': True
        }
    }
}

In [11]:
ingest_fpl.run(env='dev', overrides=overrides)

Overriding default config
Extracting data from api: https://fantasy.premierleague.com/api
Uploading table: fixtures, without schema. No schema found at: schemas/fixtures.json
Filtering first 100 rows of dataframe to autodetect schema
Uploading dataframe to bucket: gs://fpl-staging
Uploading gcsfile from gs://fpl-staging/20230511:TEMP_AD_fixtures to bigquery table: fpl_raw:TEMP_AD_fixtures
Uploading data from gs://fpl-staging/20230511:TEMP_AD_fixtures to table TEMP_AD_fixtures
Dropping table used for autodetect TEMP_AD_fixtures


## Run One-Off Load with Full Dataset

After making relevant changes to schema files, make a one-off load of the full datasets of all tables you would like to ingest. I have  included the element_types and teams tables for a one-off load, these tables do not require daily updates since they do not change throughout the season.

In [3]:
overrides = {
    'api': {
        'endpoints': [
            {'name': 'bootstrap-static',
            'tables': ['elements', 'element_types', 'teams']},
            {'name': 'fixtures'}
        ]
    }
}

In [4]:
ingest_fpl.run(env='dev', overrides=overrides)

Overriding default config
Extracting data from api: https://fantasy.premierleague.com/api
Uploading table: elements, with schema found at: schemas/elements.json
Uploading dataframe to bucket: gs://fpl-staging
Uploading gcsfile from gs://fpl-staging/20230511:elements to bigquery table: fpl_raw:elements
Uploading data from gs://fpl-staging/20230511:elements to table elements
Uploading table: element_types, with schema found at: schemas/element_types.json
Uploading dataframe to bucket: gs://fpl-staging
Uploading gcsfile from gs://fpl-staging/20230511:element_types to bigquery table: fpl_raw:element_types
Uploading data from gs://fpl-staging/20230511:element_types to table element_types
Uploading table: teams, with schema found at: schemas/teams.json
Uploading dataframe to bucket: gs://fpl-staging
Uploading gcsfile from gs://fpl-staging/20230511:teams to bigquery table: fpl_raw:teams
Uploading data from gs://fpl-staging/20230511:teams to table teams
Uploading table: fixtures, with schema f

## Test Regular Update Process

This is the function which will run daily, test it works locally first, then you are ready to schedule daily updates.

In [5]:
ingest_fpl.run(env = 'dev')

Extracting data from api: https://fantasy.premierleague.com/api
Uploading table: elements, with schema found at: schemas/elements.json
Uploading dataframe to bucket: gs://fpl-staging
Uploading gcsfile from gs://fpl-staging/20230511:elements to bigquery table: fpl_raw:elements
Uploading data from gs://fpl-staging/20230511:elements to table elements
Uploading table: element_types, with schema found at: schemas/element_types.json
Uploading dataframe to bucket: gs://fpl-staging
Uploading gcsfile from gs://fpl-staging/20230511:element_types to bigquery table: fpl_raw:element_types
Uploading data from gs://fpl-staging/20230511:element_types to table element_types
Uploading table: teams, with schema found at: schemas/teams.json
Uploading dataframe to bucket: gs://fpl-staging
Uploading gcsfile from gs://fpl-staging/20230511:teams to bigquery table: fpl_raw:teams
Uploading data from gs://fpl-staging/20230511:teams to table teams
Uploading table: fixtures, with schema found at: schemas/fixtures.